<a href="https://colab.research.google.com/github/jlap96/TrabajoFinalRedes/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RNN - IMDB Reviews**

Clasificación de sentimiento -positivo vs negativo- usando SimpleRNN y LSTM Bidireccional

Importación de librerías y establecimiento de semillas aleatorias para que los resultados sen reproducibles. Con esto aseguramos que entrenar el modelo después produzca exactamente los mismos pesos iniciales, mismo comportamiento aleatorio


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing import sequence
import time
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix


np.random.seed(42)
tf.random.set_seed(42)

Sólo se tendrán en cuenta las 10,000 palabras más frecuentes del dataset IMDB.
Declaramos la longitud fija de las reseñas, que más adelante se usará para recortar o rellenar reseñas a una longitud fija de 200 tokens

Cargamos el dataset ya preprocesados y cada reseña está convertida en una secuencia de enteros -tokens-, donde cada entero representa una palabra según un diccionario interno.

Imprimimos la división de entrenamiento y prueba, que contiene 25,000 registros cada uno. Y mostramos los primeros tokens de la primera reseña

In [ ]:
max_features = 10000  # Número de palabras más frecuentes a considerar
maxlen = 200          # Longitud fija de las reseñas

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print("Train samples:", len(x_train))
print("Test samples:", len(x_test))
print("\nEjemplo de secuencia (primeros 20 tokens):")
print(x_train[0][:20])


Train samples: 25000
Test samples: 25000

Ejemplo de secuencia (primeros 20 tokens):
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25]


Declaramos el padding.

El padding nos sirve para que todas las reseñas tengan exactamente la misma loingitus -200 tokens-.

Si una reseña es más corta, se rellenan ceros al inicio. Si es más larga, se recorta para que quede en 200.

Esto lo necesitamos porque las redes neuronales no pueden procesar secuencias con longitudes diferentes.

Imprimimos el número de reseñas de entrenamiento y la longitud fija de cada secuencia después del padding

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print("Shape después de padding:", x_train.shape)

Shape después de padding: (25000, 200)


**Baseline**

Se define el modelo RNN

Se compila el modelo utilizando adam como optimizador, binary_crossentropy porque es una clasificación binario y accuracy como métrica principal.

Después realizamos el entrenamiento y medición del tiempo (entrena durante 5 épocas, usa lotes de 64, usa 20% del train como validación y calcula cuánto tiempo tardó en entrenarse).

Se evalua en el set de prueba.

Se registrar en un dataframe los resultados

In [ ]:
model_rnn = Sequential([
Embedding(max_features, 64, input_length=maxlen),
SimpleRNN(64),
Dense(1, activation='sigmoid')
])


model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_rnn.summary()


start = time.time()
hist_rnn = model_rnn.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)
end = time.time()
print('Tiempo(s):', end - start)

loss_r, acc_r = model_rnn.evaluate(x_test, y_test, verbose=0)
print("Baseline RNN test accuracy:", acc_r)

log_rnn = pd.DataFrame([{
    'model': 'rnn_baseline',
    'params': 'Embedding64 + SimpleRNN64',
    'epochs': 5,
    'test_accuracy': acc_r,
    'time_sec': end - start
}])

log_rnn

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 70ms/step - accuracy: 0.5789 - loss: 0.6641 - val_accuracy: 0.7260 - val_loss: 0.5389
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 68ms/step - accuracy: 0.8161 - loss: 0.4092 - val_accuracy: 0.7294 - val_loss: 0.5302
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.8852 - loss: 0.2833 - val_accuracy: 0.8092 - val_loss: 0.4478
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 67ms/step - accuracy: 0.8840 - loss: 0.2719 - val_accuracy: 0.7902 - val_loss: 0.5302
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 67ms/step - accuracy: 0.9502 - loss: 0.1336 - val_accuracy: 0.7864 - val_loss: 0.6351
Tiempo(s): 166.13897824287415
Baseline RNN test accuracy: 0.7825199961662292


,model,params,epochs,test_accuracy,time_sec
0,rnn_baseline,Embedding64 + SimpleRNN64,5,0.78252,166.138978


**Optimización del modelo.**

Se define la arquitectura.
Embedding(128) convierte cada palabra en un vector de 128 dimensiones, esto permite representar mejor el significado de las palabras.

Se implementa Bidirectional(LSTM(64)). Un LSTM -LongShort-Term Memory- (Memoria a Corto-Largo plazo) normal lee la secuencia solo hacia adelante. El Bidirectional usa dos LSTM al mismo tiempo: uno lee de izquiera a derecha y otro de derecha a izquierda. La salida final combina ambos.

Con Dropout apagamos aleatoriamente el 50% de las neuronas durante el entrenamiento, reducimos el sobreajuste y obligamos al modelo a generalizar mejor.

Con Dense(1,sigmoid) es la capa final para predecir probabilidad de sentimiento positivo.

In [ ]:
model_rnn_opt = Sequential([
    Embedding(max_features, 128, input_length=maxlen),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_rnn_opt.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_rnn_opt.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

**Entrenamiento del modelo optimizado**

Entrenamos el modelo optimizado, medimos el tiempo total y evaluamos el modelo optimizado calculando la pérdida en testo y la precisión final del modelo optimizado.

Guardamos los resultados en el DataFrame

In [ ]:
start = time.time()

hist_rnn_opt = model_rnn_opt.fit(
    x_train, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.2
)

end = time.time()
print("Tiempo (s):", end - start)

loss_ro, acc_ro = model_rnn_opt.evaluate(x_test, y_test, verbose=0)
print("Optimized RNN test accuracy:", acc_ro)

log_rnn = pd.concat([log_rnn, pd.DataFrame([{
    'model': 'rnn_optimized',
    'params': 'Embedding128 + BiLSTM64 + Dropout',
    'epochs': 5,
    'test_accuracy': acc_ro,
    'time_sec': end - start
}])], ignore_index=True)

log_rnn.to_csv("experiments_log_rnn.csv", index=False)

log_rnn


Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 139s 432ms/step - accuracy: 0.6878 - loss: 0.5495 - val_accuracy: 0.8568 - val_loss: 0.3511
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 134s 427ms/step - accuracy: 0.8827 - loss: 0.2922 - val_accuracy: 0.8654 - val_loss: 0.3260
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 144s 433ms/step - accuracy: 0.9216 - loss: 0.2098 - val_accuracy: 0.8414 - val_loss: 0.4041
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 133s 425ms/step - accuracy: 0.9348 - loss: 0.1805 - val_accuracy: 0.8506 - val_loss: 0.3955
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 144s 432ms/step - accuracy: 0.9496 - loss: 0.1481 - val_accuracy: 0.8556 - val_loss: 0.3808
Tiempo (s): 693.4146130084991
Optimized RNN test accuracy: 0.8495200276374817


,model,params,epochs,test_accuracy,time_sec
0,rnn_baseline,Embedding64 + SimpleRNN64,5,0.78252,166.138978
1,rnn_optimized,Embedding128 + BiLSTM64 + Dropout,5,0.84952,693.414613


**Generar predicciones del modelo**

Comenzamos devolviendo probabilidades entre 0 y 1 donde >=0.5 -> 1 (sentimiento positivo) y <0.5 ->0 (sentimiento negativo)

Reporta de clasificaciones utilizando accuracy, recall, f1-score y support y generamos la matriz de confusión para permitir ver cuántos errores comete, si confunde una clase más que otra y si tiene sesgo hacia positivos o negativos

In [ ]:
y_pred = (model_rnn_opt.predict(x_test) > 0.5).astype('int32')

print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)


782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 53ms/step
              precision    recall  f1-score   support

           0       0.82      0.89      0.86     12500
           1       0.88      0.81      0.84     12500

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

Confusion matrix:
 [[11173  1327]
 [ 2435 10065]]


**Identificación de los ejemplos mal clasificados**

Comenzamos recorriendo los índices del conjunto de prueba y guardando los índices donde lal predicción no coincide con la etiqueta real.

Imprimimos cuántos errores hizo el modelo y el índice del primer ejemplo mal clasiicado.

Reconstruimos el texto original decoficado tokens.

Creamos una función para convertir la secuencia de números en texto. Si la palabra no está en el diccionario usa "?"

Imprimimos una reseña mal clasificada.

Mostramos etiqueta real vs predicción

In [ ]:
mis_idx = [i for i in range(len(y_test)) if y_pred[i] != y_test[i]]

print("Total misclassified:", len(mis_idx))
print("Ejemplo índice:", mis_idx[0])

# Mostrar la reseña decodificada
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(encoded_review):
    return " ".join([reverse_word_index.get(i - 3, "?") for i in encoded_review])

print("\nReseña mal clasificada:")
print(decode_review(x_test[mis_idx[0]]))

print("\nEtiqueta real:", y_test[mis_idx[0]])
print("Modelo predijo:", y_pred[mis_idx[0]][0])


Total misclassified: 3762
Ejemplo índice: 3
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

Reseña mal clasificada:
? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? i generally love this type of movie however this time i found myself wanting to kick the screen since i can't do that i will just complain about it this was absolutely idiotic the things that happen with the dead kids are very cool but the alive people are absolute idiots i am a grown man pretty big and i can defend myself well however i would not do half the stuff the little girl does in this movie also the mother in this movie is reckless with her children to the point of neglect i wish i wasn't so angry about her and her actions because i would have otherwise enjoyed the flick what a number she was take my advise and fast forward through everything you see her do until the end also is anyone else getting sick of watching movies that are filmed so dark anymore one can hardly see what is being filmed as an audience we are ? invol